In [46]:
from helper import get_input

INPUTS = get_input(day=5)
INPUTS.splitlines()[:5]

['305611713170986-307905965847624',
 '12998381660604-15936296075106',
 '162066655737986-166911023643586',
 '510900132568076-512313213628552',
 '706001751286-1356033568084']

## Insights

For this one I figured I'd build a custom object to contain the ranges and add some helper methods.

At first I figured I would work through the list of the ranges and process them to find all the ones that overlap,
but then figured for part 1 I probably won't need to do that.

In [47]:
from typing import Self

from dataclasses import dataclass


@dataclass
class FreshRange:
    start: int
    end: int

    def __contains__(self, val: int) -> bool:
        return self.start <= val <= self.end

    def __lt__(self, other) -> bool:
        return self.start < other.start

    def __hash__(self):
        """Returns the hash of a tuple of `start` and `end` for this range."""
        return tuple.__hash__((self.start, self.end))

    @classmethod
    def are_overlapping(cls, range1: Self, range2: Self) -> bool:
        """Whether the ranges overlap values with each other.

        Tests check if either the start or end values of either range are contained "in" the other.
        """
        return (
            range1.start in range2
            or range1.end in range2
            or range2.start in range1
            or range2.end in range1
        )

    @classmethod
    def merge_ranges(cls, range1: Self, range2: Self) -> None:
        """Combines two mergeable ranges into a single range, with lowest start and highest end values.

        First tests if the ranges overlap using `are_overlapping()`. If not, raises `ValueError`.
        """
        if not cls.are_overlapping(range1, range2):
            raise ValueError("Can't merge them!")

        # Preserve range1 with the values of both ranges combined
        range1.start = min(range1.start, range2.start)
        range1.end = max(range1.end, range2.end)

        # Then, set the start and end of the other range to 0, eliminating that range content
        range2.start, range2.end = 0, 0

    @classmethod
    def from_input(cls, val: str) -> Self:
        start, end = val.strip().split("-")
        return cls(start=int(start), end=int(end))

    def num_valid(self) -> int:
        if self.start == self.end:
            return 1 if self.start else 0
        return self.end + 1 - self.start

    def backwards(self):
        return self.end < self.start

In [48]:
type FreshRanges = list[FreshRange]
type Ingredients = list[int]


def prep_inputs(inputs: str) -> tuple[FreshRanges, Ingredients]:
    ranges, ingredients = inputs.split("\n\n")
    ranges = [FreshRange.from_input(x.strip()) for x in ranges.split("\n")]
    ranges = [x for x in ranges if x.num_valid()]
    ingredients = list(map(int, ingredients.strip().split("\n")))
    return (ranges, ingredients)


INPUT_RANGES, AVAILABLE_INGREDIENTS = prep_inputs(INPUTS)

INPUT_RANGES[:5], AVAILABLE_INGREDIENTS[:5]

([FreshRange(start=305611713170986, end=307905965847624),
  FreshRange(start=12998381660604, end=15936296075106),
  FreshRange(start=162066655737986, end=166911023643586),
  FreshRange(start=510900132568076, end=512313213628552),
  FreshRange(start=706001751286, end=1356033568084)],
 [28061396593815,
  234836531376214,
  92542048892680,
  518309518327845,
  557909974093724])

In [49]:
TEST_INPUTS = """
3-5
10-14
16-20
12-18

1
5
8
11
17
32
""".strip()

TEST_INPUT_RANGES, TEST_AVAILABLE_INGREDIENTS = prep_inputs(TEST_INPUTS)
print(TEST_INPUT_RANGES, TEST_AVAILABLE_INGREDIENTS, sep="\n")

[FreshRange(start=3, end=5), FreshRange(start=10, end=14), FreshRange(start=16, end=20), FreshRange(start=12, end=18)]
[1, 5, 8, 11, 17, 32]


## Part 1

Pretty straightforward. Start a search through the ranges to find one that contains the given `ingredient`, and if so, add 1 to our result.
Crucially, `break` from that inner loop to prevent searching exhaustively.

In [50]:
def part1(ranges: FreshRanges, ingredients: Ingredients) -> int:
    result = 0
    for ingredient in ingredients:
        for this_range in ranges:
            if ingredient in this_range:
                result += 1
                break
    return result


print("EXPECTED:", 3)
print(
    "TEST:    ", part1(ranges=TEST_INPUT_RANGES, ingredients=TEST_AVAILABLE_INGREDIENTS)
)
print("REAL:    ", part1(ranges=INPUT_RANGES, ingredients=AVAILABLE_INGREDIENTS))

EXPECTED: 3
TEST:     3
REAL:     773


## Part 2

This seems pretty simple, given the object we've built up already. However, we do need to be careful about those overlapping ranges to avoid double counting.

At first I thought I would merge those ranges and somehow destroy one, but that ends up being a bit wonky.
I think I can still do that, but I also need to adjust one of the two ranges to start and end at `0`, so it contains nothing.

In [51]:
from itertools import combinations
import copy


def make_ranges_exclusive(ranges: FreshRanges) -> FreshRanges:
    these_ranges = copy.deepcopy(ranges)
    for range1, range2 in combinations(these_ranges, 2):
        if not range1.num_valid() or not range2.num_valid():
            # One of the ranges is all 0s, which happens when we alter one in a merge.
            # Skip this combo
            continue
        if FreshRange.are_overlapping(range1, range2):
            FreshRange.merge_ranges(range1, range2)
    these_ranges = sorted(these_ranges)
    if not these_ranges[0].num_valid():
        # Something with a 0 value is present.
        these_ranges = [x for x in these_ranges if x.num_valid()]
        return make_ranges_exclusive(ranges=these_ranges)
    return these_ranges

In [52]:
print(">> Original:")
print(TEST_INPUT_RANGES)
print(">> Reduced:")
print(make_ranges_exclusive(TEST_INPUT_RANGES))

>> Original:
[FreshRange(start=3, end=5), FreshRange(start=10, end=14), FreshRange(start=16, end=20), FreshRange(start=12, end=18)]
>> Reduced:
[FreshRange(start=3, end=5), FreshRange(start=10, end=20)]


In [53]:
print("VALIDATION:")
print(
    part1(
        ranges=make_ranges_exclusive(TEST_INPUT_RANGES),
        ingredients=TEST_AVAILABLE_INGREDIENTS,
    )
)

VALIDATION:
3


In [54]:
def part2(ranges: FreshRanges):
    return sum(x.num_valid() for x in make_ranges_exclusive(ranges=ranges))


print("EXPECTED:", 14)
print("TEST:    ", part2(ranges=TEST_INPUT_RANGES))
print("REAL:    ", part2(ranges=INPUT_RANGES))

EXPECTED: 14
TEST:     14
REAL:     332067203034711


After many off-by-1 errors and head scratching, I finally came away with a good result!

Along the way I discovered some of the ranges end up "backwards"? They somehow have `end` values lower than their `starts`, and I can't understand why.

Oh well, leave it be.